In [1]:
import os
import re
import glob
import yaml
import json

import numpy as np
import pandas as pd

from math import ceil
from scipy.stats import entropy

import matplotlib.pyplot as plt
from matplotlib import cm, colors
from matplotlib.patches import Patch

import zarr
import napari
import tifffile
import dask.array as da

from utils.utility_functions import single_channel_pyramid

plt.rcParams['font.family'] = 'Arial'

In [2]:
clustering = 'VAE9_VIG7'

# specify tumor cell clusters
tumor = [0, 1, 3, 5, 6, 21, 22]

In [3]:
# I/O
sample = 'CRC-097'

# read single-cell data
main = pd.read_csv(os.path.join(os.getcwd(), 'input/main.csv'))
main = main[~main.isnull().any(axis=1)]  # removing cells for which KNN classes were not defined

# read OME-TIFF, segmentation outlines, and H&E channels
tif_path = os.path.join(os.getcwd(), f'input/{sample}_image.ome.tif')
seg_path = os.path.join(os.getcwd(), f'input/{sample}_seg_outlines.ome.tif')
he_path = os.path.join(os.getcwd(), f'input/{sample}_hema_eosin.ome.tif')

# import markers.csv
markers = pd.read_csv(os.path.join(os.getcwd(), 'input/CRC-097_mcmicro_markers.csv'))

# get markers excluded from analysis
with open(os.path.join(os.getcwd(), 'input/CRC-97_cylinter_config.yml')) as f:
    config = yaml.safe_load(f)
markers_to_exclude = config['markersToExclude']

# import image contrast settings
with open(os.path.join(os.getcwd(), 'input/CRC-097_cylinter_contrast_limits.yml')) as f:
    contrast_limits = yaml.safe_load(f)

# the parquet file at the path below is being read because "main.csv" 
# uses trimmed marker channel names as column headers that differ from the raw channel names used 
# in the markers.csv file, which is itself used to index channels in the OME-TIFF image.
for_channels = pd.read_parquet(
    os.path.join(os.getcwd(), 'input/CRC-097_clean_cylinter_clustering_3d_leiden.parquet')
)

# isolate antibodies of interest
abx_channels = [
    i for i in for_channels.columns if 'nucleiRingMask' in i if
    'Hoechst' not in i if i not in markers_to_exclude
]

# get name of first DNA channel
dna1 = markers['marker_name'][markers['channel_number'] == 1][0]
dna_moniker = str(re.search(r'[^\W\d]+', dna1).group())

out = os.path.join(os.getcwd(), f'output/knn/{clustering}')
if not os.path.exists(out):
    os.makedirs(out)

In [4]:
# assign clusters to KNN classes
cluster_class_dict = {}
for cluster, group in main.groupby(clustering):

    means = []

    for cls in ['Prob_Normal', 'Prob_Glandular', 'Prob_Solid', 'Prob_Mucin']:
        means.append(group[cls].mean())
    
    max_value = np.argmax(means)
    
    if max_value == 0:
        cluster_class_dict[cluster] = ('Normal', means)
    elif max_value == 1:
        cluster_class_dict[cluster] = ('Glandular', means)
    elif max_value == 2:
        cluster_class_dict[cluster] = ('Solid', means)
    elif max_value == 3:
        cluster_class_dict[cluster] = ('Mucinous', means)

# convert dict to dataframe and save
cluster_class_df = pd.DataFrame.from_dict(cluster_class_dict).T

probs = pd.DataFrame(
    cluster_class_df[1].to_list(),
    columns=['Prob_Normal', 'Prob_Glandular', 'Prob_Solid', 'Prob_Mucin']
).astype('float32')
cluster_class_df.drop(columns=1, inplace=True)
cluster_class_df.reset_index(inplace=True)
cluster_class_df = pd.concat([cluster_class_df, probs], axis=1)
cluster_class_df.rename(columns={'index': clustering, 0: 'KNN_class'}, inplace=True)

cluster_class_df = cluster_class_df[cluster_class_df[clustering].isin(tumor)]
cluster_class_df.reset_index(drop=True, inplace=True)
cluster_class_df.to_csv(os.path.join(out, f'{clustering}_KNN_classes.csv'), index=False)

In [5]:
# compute kNN class entropy scores 
entropies = {}
rows = []
for cluster, group in main.groupby(clustering):
    if cluster in tumor:
        means = []
        for cls in ['Prob_Solid', 'Prob_Mucin', 'Prob_Glandular']:
            means.append(group[cls].mean())
        entropies[cluster] = entropy(pk=means, base=2)
        data_dict = {header: [value] for header, value in zip(['Prob_Solid', 'Prob_Mucin', 'Prob_Glandular'], means)}
        row = pd.DataFrame(data_dict, index=[cluster])
        rows.append(row)
bar = pd.concat(rows, ignore_index=False)

In [6]:
# normalize mean cluster kNN posterior probabilites scores 0-1, sort clusters by entropy

# sort entropy dict by entropy values
entropies = dict(sorted(entropies.items(), key=lambda item: item[1]))

# pretty print the dictionary
print('Cluster entropies:')
print(json.dumps(entropies, indent=4))
sd = round(np.std(list(entropies.values()), ddof=1), 2)
print(f'Standard deviation = {sd}')

bar = bar.div(bar.sum(axis=1), axis=0)
bar = bar.reindex(entropies.keys())

# rename kNN class labels
bar.columns = [i.split('_')[1][0].lower() + i.split('_')[1][1:] for i in bar.columns]

Cluster entropies:
{
    "21": 0.3675085845641414,
    "0": 0.6439152699550817,
    "22": 0.9773107590248317,
    "3": 1.0191502563220842,
    "5": 1.1041689819331137,
    "6": 1.2670749930838991,
    "1": 1.3664907916049898
}
Standard deviation = 0.35
--Return--
None
> /var/folders/_h/pbzrx8ss6n5f031pf4hc97_w0000gp/T/ipykernel_17954/734805608.py(14)<module>()
     12 bar = bar.div(bar.sum(axis=1), axis=0)
     13 bar = bar.reindex(entropies.keys())
---> 14 import pdb; pdb.set_trace()
     15 # rename kNN class labels
     16 bar.columns = [i.split('_')[1][0].lower() + i.split('_')[1][1:] for i in bar.columns]



ipdb>  bar


    Prob_Solid  Prob_Mucin  Prob_Glandular
21    0.041974    0.015811        0.942215
0     0.037312    0.084825        0.877864
22    0.085355    0.136698        0.777947
3     0.142595    0.094157        0.763248
5     0.108286    0.161890        0.729824
6     0.070913    0.564594        0.364493
1     0.522981    0.107945        0.369073


ipdb>   0.522981+0.107945+0.369073


0.999999


ipdb>  exit


In [ ]:
# plot
fig, ax1 = plt.subplots()  # figsize=(11, 5)
ax2 = ax1.twinx()
x = bar.index.astype('str')
indexes = np.argsort(bar.values).T
heights = np.sort(bar.values).T
order = -1
bottoms = heights[::order].cumsum(axis=0)
bottoms = np.insert(bottoms, 0, np.zeros(len(bottoms[0])), axis=0)

# make color dict
# normal = ['normal'] 
solid = ['solid']
mucin = ['mucin']
glandular = ['glandular']
mpp_colors = {}
# mpp_colors.update({str(k): v for k, v in zip(normal, ['tab:blue']*len(normal))})
mpp_colors.update({str(k): v for k, v in zip(solid, ['tab:red']*len(solid))})
mpp_colors.update({str(k): v for k, v in zip(mucin, ['tab:blue']*len(mucin))})
mpp_colors.update({str(k): v for k, v in zip(glandular, ['tab:green']*len(glandular))})

for btms, (idxs, vals) in enumerate(list(zip(indexes, heights))[::order]):

    mps = np.take(np.array(bar.columns), idxs)

    # IF YOU DONT WANT TO COLOR THE BARS, USE THIS CODE
    # g = ax1.bar(
    #     x, height=vals, width=0.9, color='w', lw=0.1, ec='k',
    #     bottom=bottoms[btms]
    #     )

    g = ax1.bar(
        x, height=vals, width=0.9, lw=0.5, bottom=bottoms[btms],
        color=[mpp_colors[m] for m in mps], alpha=0.5
        )
    
    # ANNOTATE BARS 
    # pos = -1
    # for i, b in enumerate(g.patches):
    #     if i % len(bar.index) == 0:
    #         pos += 1
    #     xloc = b.get_x() + 0.45
    #     yloc = b.get_y() + b.get_height()/2
    #     if b.get_height() > 0.05:
    #         ax1.annotate(
    #             mps[i], xy=(xloc, yloc), fontname='Arial',
    #             va='center', ha='center', size=9
    #             )

ax1.set_ylim(0, 1.0)
ax1.set_xlabel('Tumor-associated VAE cluster', size=15, labelpad=10)
ax1.set_ylabel('Normalized posterior probability', size=13, labelpad=13, c='k')
ax1.tick_params(axis='x', which='major', labelsize=11)
ax1.tick_params(axis='y', which='major', labelsize=11)
ax1.margins(x=0)

patches = [
    Patch(facecolor=color, alpha=0.5, edgecolor=None) for 
    color in ['tab:red', 'tab:blue', 'tab:green', ]
    ]

plt.legend(
    patches, ['solid', 'mucinous', 'glandular'],
    title=None, prop={'size': 12},
    labelspacing=0.01, loc='upper left'
    ) # bbox_to_anchor=[-0.8, 1.0], 

ax2.plot(
    [str(i) for i in entropies.keys()], entropies.values(),
    c='tab:red', lw=3, linestyle='--'
    )
ax2.set_ylabel('Class entropy (--)', size=13, labelpad=13, c='k')
ax2.tick_params(axis='both', which='major', labelsize=11)

plt.tight_layout()
fig.savefig(os.path.join(out, 'kNN_class_entropy.pdf'))
print('There is a spatial pattern from left to right in the image going from glandular to mucinous to solid tumor.')
plt.show()
plt.close('all')

In [ ]:
# view kNN posterior probability scores and cluster class entropies in image viewer

# add H&E image (separate RGB channels)
for color, channel in zip(['red', 'green', 'blue'], [0, 1, 2]):

    img, min, max = single_channel_pyramid(glob.glob(he_path)[0], channel=channel)

    if channel == 0:
        viewer = napari.view_image(
            img, rgb=False, colormap=color, blending='additive',
            visible=False, name=f'H&E_{color}'
        )
    else:
        viewer.add_image(
            img, rgb=False, colormap=color, blending='additive',
            visible=False, name=f'H&E_{color}'
        )

# read DNA1 channel
dna, min, max = single_channel_pyramid(glob.glob(tif_path)[0], channel=0)

# add DNA1 channel to Napari image viewer
viewer.add_image(
    dna, rgb=False, blending='additive', colormap='gray',
    visible=True, opacity=1.0, name='DNA1', contrast_limits=(min, max)
)

# loop over antibodies of interest and add to image viewer
for ch in abx_channels:
    ch = ch.rsplit('_', 1)[0]
    channel_number = markers['channel_number'][markers['marker_name'] == ch]
    img, min, max = single_channel_pyramid(
        glob.glob(tif_path)[0], channel=(channel_number.item() - 1)
    )
    viewer.add_image(
        img, rgb=False, blending='additive', colormap='green', visible=False, name=ch, contrast_limits=(min, max)
    )

# apply previously defined contrast limits
for ch in abx_channels:
    ch = ch.rsplit('_', 1)[0]
    viewer.layers[ch].contrast_limits = (
        contrast_limits[ch][0], contrast_limits[ch][1])

# map clusters to color categories and add centroids for tumor clusters to image viewer
num_colors = len(list(cm.tab20.colors))
num_clusters = len(main[clustering].unique())
palette_multiplier = ceil(num_clusters / num_colors)
_colors = list(cm.tab20.colors) * palette_multiplier
_colors = _colors[0:num_clusters]
_colors.reverse()

color_dict = dict(zip(sorted(main[clustering].unique(), reverse=True), _colors))

for cls in ['Glandular', 'Solid', 'Mucinous', 'Normal']:

    clusters = [k for k, v in cluster_class_dict.items() if v[0] == cls]
    clusters = sorted([i for i in clusters if i in tumor], reverse=True)
    
    for i in clusters:
        centroids = main[['Y_centroid', 'X_centroid']][main[clustering] == i]
        viewer.add_points(
            centroids, name=f'{clustering}_{i}_{cluster_class_dict[i][0]}',
            face_color=np.array(color_dict[i]), border_width=0.0,
            size=60.0, opacity=1.0, blending='translucent', visible=False
        )
        
        # color clusters by their class entropy
        norm = colors.Normalize(vmin=0.0, vmax=1.585)  # 0-1 normalize between range for 3 classes
        colormap = cm.viridis
        color_mapped = colormap(norm(entropies[i]))
        viewer.add_points(
            centroids, name=f'{clustering}_{i}_Entropy',
            face_color=np.array(color_mapped[:3]), border_width=0.0,
            size=60.0, opacity=1.0, blending='translucent', visible=False
        )

# colormap centroids by KNN class posterior probabilities
for cls in ['Prob_Glandular', 'Prob_Mucin', 'Prob_Solid', 'Prob_Normal']:

    centroids = main[['Y_centroid', 'X_centroid']]
    point_properties = {'probability': main[cls]}

    viewer.add_points(
        centroids, name=cls, properties=point_properties,
        face_color='probability', face_colormap='viridis',
        border_width=0.0, size=60.0, opacity=1.0, blending='translucent',
        visible=False
    )

# read segmentation outlines
seg, min, max = single_channel_pyramid(glob.glob(seg_path)[0], channel=0)
viewer.add_image(
    seg, rgb=False, blending='additive', colormap='gray', visible=False, 
    name='segmentation', opacity=1.0, contrast_limits=(min, max)
)

# run Napari image viewer
viewer.scale_bar.visible = True
viewer.scale_bar.unit = 'um'

napari.run()